In [1]:
import json
import getpass
import requests
import pandas as pd
import numpy as np
import urllib3
import datetime
import dateutil.parser
import sklearn
import matplotlib
import types
import uuid
import io

urllib3.disable_warnings()
from requests.auth import HTTPBasicAuth

from sklearn import linear_model
from sklearn.model_selection import train_test_split 
from sklearn import neighbors
from matplotlib import pyplot as plt

In [2]:
# This cell does most of the data obtaining and cleanup work. It first connects to the PI System via PI Web API 
# using basic/kerberos authentication where in it gets maximum 1000 eventframes which fit our criteria. 


""" Create API call security method
        @param security_method string:  security method to use:  basic or kerberos
        @param user_name string: The user's credentials name
        @param user_password string: The user's credentials password
"""
def read_config():
    with open('appsettings.json') as c:
        config = json.load(c)
    return config
        
def call_security_method(security_method, user_name, user_password):
    
    if security_method.lower() == 'basic':
        security_auth = HTTPBasicAuth(user_name, user_password)
        
    return security_auth

""" Method to send HTTP GET requests
        @param query:  query string to execute
        Also uses the appsettings.json file to read the username and password for Basic Authentication
"""
def get(query):
    
    data = read_config()
    username = data['Username']
    password = data['Password']
    securitymethod = data.get('AuthType', 'basic')
    verify_ssl = data.get('VerifySSL', False)
    
    security_auth = call_security_method(securitymethod, username, password)
    response = requests.get(query, auth=security_auth, verify=verify_ssl)
    return response

""" Method to send HTTP POST requests
        @param query:  query string to execute
        @param body: body of the request 
        Also uses the appsettings.json file to read the username and password for Basic Authentication
"""
def post(query, body):
    
    header = {
        'content-type': 'application/json',
        'X-Requested-With':'XmlHttpRequest'
    }
    
    data = read_config()
    username = data['Username']
    password = data['Password']
    securitymethod = data.get('AuthType', 'basic')
    verify_ssl = data.get('VerifySSL', False)
    
    security_auth = call_security_method(securitymethod, username, password)
    response = requests.post(query, auth=security_auth, verify=verify_ssl, json=body, headers=header)
    return response

""" Method to send HTTP DELETE requests
        @param query:  query string to execute
        Also uses the appsettings.json file to read the username and password for Basic Authentication
"""
def delete(query):
    data = read_config()
    username = data['Username']
    password = data['Password']
    securitymethod = data.get('AuthType', 'basic')
    verify_ssl = data.get('VerifySSL', False)

    security_auth = call_security_method(securitymethod, username, password)
    response = requests.delete(query, auth=security_auth, verify=verify_ssl)
    return response

""" Method to get the database web ID of a given database path
        @param path: path of the database. More info can be found here: https://your-server/piwebapi/help/topics/path-syntax
"""
def getAFDatabaseWebID(path):
    data = read_config()
    piwebapi_url = data['Resource']
    getAFDBQuery = piwebapi_url + "/assetdatabases?path=" + path  
    AFDB = json.loads(get(getAFDBQuery).text)
    return AFDB['WebId']

def getPIDatabaseWebID(path):
    data = read_config()
    piwebapi_url = data['Resource']
    getPIQuery = piwebapi_url + "/dataservers?path=" + path  
    PI = json.loads(get(getPIQuery).text)
    return PI['WebId']

def getPIPointWebID(pipoint):
    data = read_config()
    piwebapi_url = data['Resource']
    getPIPointQuery = piwebapi_url + "/points?path=" + pipoint  
    PIPoint = json.loads(get(getPIPointQuery).text)
    return PIPoint['WebId']

def getPIPointHistory(path):
    data = read_config()
    piwebapi_url = data['Resource']
    GetPIPointQuery = piwebapi_url + "/streams/" + path   
    GPIPoint = get(GetPIPointQuery)
    GPIPointdata = json.loads(GPIPoint.text)
    return GPIPointdata


def postPIPointValue(path, body):
    data = read_config()
    piwebapi_url = data['Resource']
    PostPIPointQuery = piwebapi_url + "/streams/" + path + "/value" 
    #print(PostPIPointQuery)
    post_response = post(PostPIPointQuery, body)
    return post_response

def postPIPointTag(path, body):
    data = read_config()
    piwebapi_url = data['Resource']
    PostPIPoint = piwebapi_url + "/dataservers/" + path + "/points" 
    #print(PostPIPointQuery)
    post_response = post(PostPIPoint, body)
    return post_response

#get AFdatabase WebID
data = read_config()
databasePath = "\\\\" + data['AFServerName'] + "\\" + data['AFDatabaseName']
AFdatabaseWebID = getAFDatabaseWebID(databasePath)
#Use this ID to make subsequent Calls to AF Database
print(AFdatabaseWebID)

#Get PI Database WebID
archivePath = "\\\\PIServers" + "[" + data['DataArchiveName'] + "]"
PIdatabaseWebID = getPIDatabaseWebID(archivePath)
#Use this ID to make subsequent Calls to PI Database
print(PIdatabaseWebID)

F1RDHkW0geEhXE-8JFJzbujJVgYUy995KW50iSI5oGHKtY2ARFNPLURFVi1BRjAxXE5FVFdPUks
F1DSN_aprwPEz0Cukryh4byYPARFNPLURFVi1QSTAx


In [3]:


#Get PI Point WebID and last Day datapoints'

#meter.10001002_E1.Current
#meter.10001002_E1.Voltage
#meter.10001002_E1.ActivePower
#meter.10001002_E1.ReactivePower

pipoint = "meter.10001002_E1.Current"
pipointPath = "\\\\" + data['DataArchiveName'] + "\\" + pipoint
PIPointWebID = getPIPointWebID(pipointPath)

#Use this ID to make subsequent Calls to PI Database for each PIPoint
#print(PIPointWebID)

#Get Last 1 day Data for PiPoint using previous WebID
PiPath = PIPointWebID + "/recorded?startTime=" + "*-5d"
PI2 = getPIPointHistory(PiPath)
PI2


KeyError: 'WebId'

In [ ]:
#Update Pi Point Timestamp and Value

pipoint_imp = "meter.10010827_E1.ImpedanceMaxCalc"
pipointPath = "\\\\" + data['DataArchiveName'] + "\\" + pipoint_imp
PIPointWebID = getPIPointWebID(pipointPath)
print(PIPointWebID)
#body = {"Timestamp": "2023-10-30T03:10:00Z", "Value": 4.9, "Good": True}
#t1 = postPIPointValue(PIPointWebID, body)
#print(t1)
#t1.headers['Location']


In [ ]:
#Get PI Database WebID
archivePath = "\\\\PIServers" + "[" + data['DataArchiveName'] + "]"
PIdatabaseWebID = getPIDatabaseWebID(archivePath)
#Use this ID to make subsequent Calls to PI Database
print(PIdatabaseWebID)
body = { "Name": "PointName1", "Descriptor": "" , "PointClass": "classic",  "PointType": "Float32", "EngineeringUnits": "", "Step": False, "Future": False, "DisplayDigits": -5 }
t1 = postPIPointTag(PIdatabaseWebID, body)
t1

In [13]:
import pyodbc as pc
import urllib
import sqlalchemy as sa

ImportError: libodbc.so.2: cannot open shared object file: No such file or directory

In [11]:
params =  urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                                "Server=DSO-DEV-SQL01;"
                                #"Database=db;"
                                  "UID= dsosqladmin;"
                                  "PWD=Letmein123!;"
                                "Trusted_Connection=yes;"
                                )
coxn = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params),fast_executemany=True)

NameError: name 'urllib' is not defined